In [1]:
import pandas as pd

In [4]:
# Read the CSV file into a data frame
data = pd.read_csv('sp500-data/A.csv')

# Print the first five rows of the data frame
data.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,18-11-1999,28.612303,32.546494,62546380,35.765381,31.473534,26.929760
1,19-11-1999,28.478184,30.713518,15234146,30.758226,28.880545,24.711119
2,22-11-1999,28.657009,29.551144,6577870,31.473534,31.473534,26.929760
3,23-11-1999,28.612303,30.400572,5975611,31.205294,28.612303,24.481602
4,24-11-1999,28.612303,28.701717,4843231,29.998213,29.372318,25.131901
